In [1]:
import sys

In [2]:
import numpy

In [3]:
import matplotlib

In [4]:
import bpy

In [5]:
from main import *
from pathlib import Path
import pandas as pd

(2805,)
(2805, 3)
-0.014483500272035599 0.01615884155035019
-2.6304002 2.795992
[ 0.00678759  0.00672449  0.0066078  ... -0.00663939 -0.00665385
 -0.00662998]
[[-2.6304002 -2.6304002 -2.495775 ]
 [-2.6304002 -2.6304002 -2.3193827]
 [-2.6304002 -2.6304002 -2.1429904]
 ...
 [ 2.6613667  2.6613667  2.4432075]
 [ 2.6613667  2.6613667  2.6195998]
 [ 2.6613667  2.6613667  2.795992 ]]


In [99]:
# This is ...
def minmax(df):
    return df.apply(lambda x: (x - x.min()) / (x.max() - x.min()))

def color(df):
    # convert esp to colors using red to blue colormap
    cmap = cm.get_cmap('terrain_r')
    norm = plt.Normalize(vmin=df.min(), vmax=df.max())
    # norm = plt.Normalize(vmin=-0.01, vmax=0.01)
    return cmap(norm(df))


In [100]:
N = 10
for i in range(1):
        rand = pd.DataFrame(np.random.random((N,3))*10)

        remove_all()
        bplt.Scatter(
                rand,
                color=color(rand[0]),
                marker_type="uv_spheres",
                radius=0.1235,
                name="points",
            )
        add_camera()
        camera_to_points()
        render(i)

/var/folders/wy/kcg4xf054_v1h2zwbhjb2g_w0000gn/T/ipykernel_45615/3378388712.py:7: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap('terrain_r')


In [101]:
downloads = Path("/Users/ericboittier/Downloads/")

filenames = list(downloads.glob("test*png"))
filenames

[PosixPath('/Users/ericboittier/Downloads/test-47.png'),
 PosixPath('/Users/ericboittier/Downloads/test-53.png'),
 PosixPath('/Users/ericboittier/Downloads/test-84.png'),
 PosixPath('/Users/ericboittier/Downloads/test-90.png'),
 PosixPath('/Users/ericboittier/Downloads/test-91.png'),
 PosixPath('/Users/ericboittier/Downloads/test-85.png'),
 PosixPath('/Users/ericboittier/Downloads/test-52.png'),
 PosixPath('/Users/ericboittier/Downloads/test-46.png'),
 PosixPath('/Users/ericboittier/Downloads/test-50.png'),
 PosixPath('/Users/ericboittier/Downloads/test-44.png'),
 PosixPath('/Users/ericboittier/Downloads/test-78.png'),
 PosixPath('/Users/ericboittier/Downloads/test-93.png'),
 PosixPath('/Users/ericboittier/Downloads/test-87.png'),
 PosixPath('/Users/ericboittier/Downloads/test-86.png'),
 PosixPath('/Users/ericboittier/Downloads/test-92.png'),
 PosixPath('/Users/ericboittier/Downloads/test-79.png'),
 PosixPath('/Users/ericboittier/Downloads/test-45.png'),
 PosixPath('/Users/ericboittier

In [102]:
import imageio
with imageio.get_writer(downloads / 'movie.gif', mode='I') as writer:
    for filename in filenames:

        image = imageio.imread(filename)
        for i in range(60):
            writer.append_data(image)

/var/folders/wy/kcg4xf054_v1h2zwbhjb2g_w0000gn/T/ipykernel_45615/4233054693.py:5: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  image = imageio.imread(filename)


In [103]:
downloads / 'movie.gif'

PosixPath('/Users/ericboittier/Downloads/movie.gif')

![SegmentLocal](/Users/ericboittier/Downloads/movie.gif "segment")


In [153]:
# plot spherical harmonic in blender using the python api
import bpy
import numpy as np
from scipy.special import sph_harm
import mathutils

def color_mesh():
    # Ensure an object is selected and it is a mesh
    norm = plt.Normalize(vmin=-1, vmax=1)
    cmap = matplotlib.colormaps['rainbow']
    obj = bpy.context.object
    if obj and obj.type == 'MESH':
        mesh = obj.data

        # Create a vertex color layer if it doesn't exist
        if not mesh.vertex_colors:
            vertex_color_layer = mesh.vertex_colors.new(name="Col")
        else:
            vertex_color_layer = mesh.vertex_colors.active

        # Start by entering Object Mode
        bpy.ops.object.mode_set(mode='OBJECT')

        # Set the color of each vertex
        i = 0
        for poly in mesh.polygons:
            for idx in poly.loop_indices:
                # get xyz position of the vertex
                x, y, z = mesh.vertices[mesh.loops[idx].vertex_index].co
                r, theta, phi = getPolar(x, y, z)
                Amp = sph_harm(m, l, phi, theta).imag if m<0 \
                        else sph_harm(m, l, phi, theta).real
                # convert r (radius) to color
                color = cmap(norm(Amp))
                # print(Amp, color)
                r, g, b = color[0], color[1], color[2]
                # r, g, b = x, y, z  # Red color
                # r, g, b = np.random.random(1), np.random.random(1), np.random.random(1)  # Red color
                vertex_color_layer.data[i].color = (r, g, b, 1.0)
                i += 1

        # Ensure the object has a material
        if not obj.material_slots:
            # create a new material
            mat = bpy.data.materials.new(name="VertexColorMaterial")
            obj.data.materials.append(mat)
        else:
            mat = obj.material_slots[0].material

        # Enable 'Use Nodes'
        mat.use_nodes = True
        nodes = mat.node_tree.nodes

        # Clear all nodes to start clean
        nodes.clear()

        # Create a Principled BSDF shader node
        shader_node = nodes.new(type='ShaderNodeBsdfPrincipled')

        # Create a Vertex Color node and link it to the Principled BSDF
        vertex_color_node = nodes.new(type='ShaderNodeVertexColor')
        vertex_color_node.layer_name = vertex_color_layer.name  # use the correct vertex color layer
        mat.node_tree.links.new(vertex_color_node.outputs['Color'], shader_node.inputs['Base Color'])

        # Add an Output node and link it to the shader
        output_node = nodes.new('ShaderNodeOutputMaterial')
        mat.node_tree.links.new(shader_node.outputs['BSDF'], output_node.inputs['Surface'])

        # Update the mesh
        mesh.update()
    else:
        print("Selected object is not a mesh or no object is selected.")


def color(color="RED"):
    obj =  bpy.context.active_object
    # obj.name = str(l)+"_"+str(m)
    mat = bpy.data.materials.new(color)
    mat.use_nodes = True
    tree = mat.node_tree
    nodes = tree.nodes
    # bsdf = nodes["Principled BSDF"]
    # Get the Principled BSDF node
    bsdf = nodes.get("Principled BSDF")
    if bsdf is not None:
        # Set the color to black
        bsdf.inputs['Base Color'].default_value = (0.0, 0.0, 0.0, 1)
    else:
        print("Principled BSDF node not found")
    mat.diffuse_color = (float(.5),0.0,1.0,1)
    obj.active_material = mat

def getCart(r, theta, phi):
    x = r * np.cos(phi) * np.sin(theta)
    y = r * np.sin(phi) * np.sin(theta)
    z = r * np.cos(theta)
    return (x, y, z)

def getPolar(x, y, z):
    # Round out small errors from the trigonometric functions
    x, y, z = round(x, 5), round(y, 5), round(z, 5)

    r = np.sqrt(x**2 + y**2 + z**2)
    theta = np.arccos(z/r) #[0,pi]
    xy = round(r * np.sin(theta), 5)
    if xy == 0.0:
        phi = 0.0
    else:
        if x >= 0:
            phi = np.arcsin(y/xy) #[-1/2pi, 1/2pi]
        else:# x < 0:
            phi = np.arcsin(y/xy) #[-1/2pi, 1/2pi]
            phi = np.pi - phi
    return (r, theta, phi)

def camera_to_points(object_name="3_1"):
    #  set camera to fit all points
    bpy.ops.object.select_all(action="DESELECT")
    bpy.data.objects[object_name].select_set(True)
    bpy.ops.view3d.camera_to_view_selected()

In [177]:

l, m = 8,1
radius = 1

def plot_sph(l, m, radius):
    remove_all()

    # add a sphere
    bpy.ops.mesh.primitive_uv_sphere_add(segments=128, ring_count=128, \
                   radius=radius, calc_uvs=True, enter_editmode=False, \
                   align='WORLD', location=(0.0, 0.0, 0.0), \
                   rotation=(0.0, 0.0, 0.0), scale=(1.,1.,1.))
    bpy.ops.transform.translate(value=(0,0,0))

    # add material to object
    bpy.ops.object.material_slot_add()
    obj =  bpy.context.active_object
    obj.name = str(l)+"_"+str(m)

    # color the mesh
    color()
    color_mesh()

    # Make the basis shape key at minimum amplitude
    sk_basis = obj.shape_key_add(name='Basis')
    sk_basis.interpolation = 'KEY_LINEAR'
    obj.data.shape_keys.use_relative = True#False
    # The next shape key is at max amplitude
    sk2 = obj.shape_key_add(name='Deform')
    sk2.interpolation = 'KEY_LINEAR'

    for i in range(len(obj.data.vertices)):
        x, y, z = obj.data.vertices[i].co
        r, theta, phi = getPolar(x, y, z)
        Amp = sph_harm(m, l, phi, theta).imag if m<0 \
                else sph_harm(m, l, phi, theta).real

        VertMin = mathutils.Vector(getCart(r - Amp, theta, phi))
        VertMax = mathutils.Vector(getCart(r + Amp, theta, phi))
        sk_basis.data[i].co = VertMin
        sk2.data[i].co = VertMax

    #create bounding box around the shape
    bpy.ops.mesh.primitive_cube_add(size=3*radius, enter_editmode=False,
                                    align='WORLD', location=(0.0, 0.0, 0.0), scale=(1, 1, 1))
    # make the cube wireframe
    bpy.ops.object.modifier_add(type='WIREFRAME')
    bpy.context.object.modifiers["Wireframe"].thickness = 0.05

    # color_mesh()
    color(color="BLUE")
    add_camera()
    camera_to_points(object_name="Cube")
    # select everything except the camera
    bpy.ops.object.select_all(action='DESELECT')
    bpy.data.objects["Cube"].select_set(True)
    bpy.data.objects[f"{l}_{m}"].select_set(True)
    # rotate the camera for an isomorphic view
    # bpy.ops.transform.rotate(value=45, orient_axis='Y', orient_type='GLOBAL')
    # bpy.ops.transform.rotate(value=45, orient_axis='X', orient_type='GLOBAL')
    # bpy.ops.transform.rotate(value=45, orient_axis='Z', orient_type='GLOBAL')
    # bpy.ops.transform.translate(value=(0, 0, 1), orient_type='GLOBAL')
    camera_to_points(object_name="Cube")
    render(f"{l}_{m}")


In [178]:
list(bpy.data.objects)

[bpy.data.objects['0_0'], bpy.data.objects['Camera'], bpy.data.objects['Cube']]

In [181]:
for l in range(0, 8):
    if l == 0:
        for m in range(0, 1):
            plot_sph(l, m, radius)
    else:
        for m in range(-l, l+1):
            plot_sph(l, m, radius)


In [167]:
?bpy.ops.mesh.primitive_uv_sphere_add

In [ ]:
import bpy
import bmesh
import math
import numpy as np
from scipy.special import sph_harm

def create_spherical_harmonic_mesh(l, m, resolution):
    remove_all()
    mesh = bpy.data.meshes.new('SphericalHarmonicMesh')
    obj = bpy.data.objects.new('SphericalHarmonic', mesh)
    bpy.context.collection.objects.link(obj)
    bm = bmesh.new()

    for i in range(resolution):
        theta = math.pi * (i / resolution)
        for j in range(2 * resolution):
            phi = 2 * math.pi * (j / (2 * resolution))
            # Using the real part of the spherical harmonics
            r = abs(sph_harm(m, l, phi, theta).real)
            x = r * math.sin(theta) * math.cos(phi)
            y = r * math.sin(theta) * math.sin(phi)
            z = r * math.cos(theta)
            bm.verts.new((x, y, z))

    bm.verts.ensure_lookup_table()
    for i in range(len(bm.verts) - 2 * resolution):
        for j in range(2 * resolution - 1):
            try:
                v1 = bm.verts[i * 2 * resolution + j]
                v2 = bm.verts[i * 2 * resolution + j + 1]
                v3 = bm.verts[(i + 1) * 2 * resolution + j]
                v4 = bm.verts[(i + 1) * 2 * resolution + j + 1]
                bm.faces.new((v1, v2, v4, v3))
            except:
                pass

    bm.to_mesh(mesh)
    bm.free()

# Example usage: visualize the real part of the spherical harmonic Y_2^1
create_spherical_harmonic_mesh(2, 1, 100)
